<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1Predict_by_AutoML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret==2.3.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.7 MB 46.0 MB/s 
     |████████████████████████████████| 1.1 MB 43.3 MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=e00f3b0b024bd07925907c6cb31a254bfd9e207c529546ab74f98ca7a40ad23a
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
  Created wheel for imagehash: filename=ImageHash-4.2.1-py2.py3-none-any.whl size=295207 sha256=9b2d4f86b7a46a0950466d3f88c9d9fcea3c2f61910474b6db46449e387585db
  Stored in directory: /root/.cache/pip/wheels/4c/d5/59/5e3e297533ddb09407769762985d134135064c6831e29a914e
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158172 sha256=d984c2db37aa0d4835f4812d81bc5f42ff7aaaa395dcf8bb491746a7611bb269
  Stored in directory: /root/.

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


In [ ]:
import numpy as np
import pandas as pd
import time

csv = '/content/scrapeData20210812.csv'
data = pd.read_csv(csv)
data.shape

(454, 49)

# データの加工

In [ ]:
data['総移動距離'] = data['総移動距離'].str.replace('km', '').astype(np.float64)
data['相手との移動距離差'] = data['相手との移動距離差'].str.replace('km', '').astype(np.float64)
data['チャンス構築率'] = data['チャンス構築率'].str.replace('%', '').astype(np.float64)
data['成功率'] = data['成功率'].str.replace('%', '').astype(np.float64)
data['被成功率'] = data['被成功率'].str.replace('%', '').astype(np.float64)
data['被チャンス構築率'] = data['被チャンス構築率'].str.replace('%', '').astype(np.float64)
dataset = data
dataset

,Unnamed: 0,試合日,チーム名,スコア,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート.1,チャンス構築率,ゴール.1,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール.2,被ゴール期待値,被ゴール.1,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール.3,シュート.2,枠内シュート,パス.1,クロス.1,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル.1,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
0,home,2021-8-9-15:00,札幌,2,17.92,14.05,1.81,2.06,7.60,3.51,106.72,11.88,0.40,119.1,14.6,12.3,1.3,8.7,120.4,10.8,9.0,1.3,12.2,1.532,1.3,1.383,1.3,113.441,-1.966,170.5,-12.0,1.3,14.6,4.2,534.1,15.5,9.3,0.6,6.0,21.3,15.4,19.4,16.6,1.5,2.0,1.0,0.1,43.0,15.0
1,away,2021-8-9-15:00,浦和,1,15.64,13.14,1.38,1.12,5.70,2.92,91.95,13.38,0.38,110.6,9.9,9.0,1.2,11.8,107.6,13.0,12.0,1.1,8.4,1.231,1.2,1.198,1.1,114.891,-1.786,164.5,-8.7,1.2,9.9,3.7,581.6,13.2,10.2,0.7,4.7,19.2,9.4,18.8,19.0,1.5,1.2,0.8,0.0,40.2,10.8
2,home,2021-4-16-19:00,札幌,1,17.92,14.05,1.81,2.06,7.60,3.51,106.72,11.88,0.40,119.1,14.6,12.3,1.3,8.7,120.4,10.8,9.0,1.3,12.2,1.532,1.3,1.383,1.3,113.441,-1.966,170.5,-12.0,1.3,14.6,4.2,534.1,15.5,9.3,0.6,6.0,21.3,15.4,19.4,16.6,1.5,2.0,1.0,0.1,43.0,15.0
3,away,2021-4-16-19:00,横浜FM,3,21.59,17.95,2.46,1.18,9.57,4.60,103.68,11.16,0.38,128.0,16.4,12.8,1.9,11.4,123.5,10.6,8.6,0.9,8.6,1.919,1.9,1.241,0.9,121.445,1.338,214.0,22.2,1.9,16.4,5.7,664.4,21.4,11.2,3.0,6.2,24.7,10.1,18.2,16.7,1.6,1.9,1.3,0.0,58.5,17.4
4,home,2021-2-27-14:00,札幌,5,17.92,14.05,1.81,2.06,7.60,3.51,106.72,11.88,0.40,119.1,14.6,12.3,1.3,8.7,120.4,10.8,9.0,1.3,12.2,1.532,1.3,1.383,1.3,113.441,-1.966,170.5,-12.0,1.3,14.6,4.2,534.1,15.5,9.3,0.6,6.0,21.3,15.4,19.4,16.6,1.5,2.0,1.0,0.1,43.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,away,2021-5-8-14:00,広島,0,16.63,13.16,1.70,1.77,7.13,3.25,98.65,13.09,0.22,121.0,13.5,11.2,1.1,8.0,119.4,9.5,7.9,1.0,10.1,1.389,1.1,1.016,1.0,116.732,-1.146,178.5,1.3,1.1,13.5,4.3,499.2,15.0,10.4,1.5,5.7,23.8,13.7,18.5,18.3,1.8,2.1,0.6,0.0,45.7,13.2
450,home,2021-3-14-15:00,大分,1,12.18,10.48,1.23,0.46,4.09,1.57,88.49,15.57,0.55,110.1,7.3,6.6,0.6,8.3,111.5,12.6,11.3,1.4,11.4,0.843,0.6,1.404,1.4,122.001,2.745,168.1,-4.0,0.6,7.3,2.3,527.3,13.0,7.4,1.2,3.1,21.3,5.5,18.1,24.4,1.7,1.2,0.6,0.0,28.9,6.7
451,away,2021-3-14-15:00,FC東京,1,13.58,11.04,1.44,1.10,6.11,3.80,90.11,14.05,0.34,119.3,11.0,9.3,1.3,12.2,118.6,13.3,11.2,1.3,10.2,1.145,1.3,1.340,1.3,111.544,-2.040,189.2,11.7,1.3,11.0,3.4,409.9,11.7,12.3,1.2,4.9,22.3,10.3,18.4,19.7,1.2,1.3,1.0,0.0,33.5,11.0
452,home,2021-5-2-14:00,大分,1,12.18,10.48,1.23,0.46,4.09,1.57,88.49,15.57,0.55,110.1,7.3,6.6,0.6,8.3,111.5,12.6,11.3,1.4,11.4,0.843,0.6,1.404,1.4,122.001,2.745,168.1,-4.0,0.6,7.3,2.3,527.3,13.0,7.4,1.2,3.1,21.3,5.5,18.1,24.4,1.7,1.2,0.6,0.0,28.9,6.7


In [ ]:
# data = dataset.sample(frac=0.9, random_state=786)  # 訓練データ
# data_unseen = dataset.drop(data.index)  # 未知のデータ

# data.reset_index(drop=True, inplace=True)  # インデックスを初期化
# data_unseen.reset_index(drop=True, inplace=True)  # インデックスを初期化

# print("訓練データの形状: " + str(data.shape))
# print("未知のデータの形状: " + str(data_unseen.shape))

In [ ]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}
pred_team = {'FC東京':'270','Ｇ大阪':'128'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

# get Pred Data

In [ ]:
#試合日は以下を例に入力
#2021-8-9-15:00
def scrapePredData(homeTeamID,awayTeamID,year,matchDate):
    data = pd.DataFrame()
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #チーム毎の平均スタッツを取得
    #チャンスビルディングポイント
    df = pd.read_html(url)[12]
    df = df.drop([0,2,4,6,8,10,12,14,16]).drop(df.columns[[0,1,5,6]], axis=1)
    # #チームスタイル
    #指数
    # df2 = pd.read_html(url)[13]
    # df2 = df2.drop([0,2,4,6,8,10,12,14]).drop(df2.columns[[0,1,5,6]], axis=1)
    #チャンス構築率
    df3 = pd.read_html(url)[16]
    df3 = df3.drop([0,2,4,6,8,10,11,13,15,17,19]).drop(df3.columns[[0,1,5,6]], axis=1)
    #ゴール期待値
    df4 = pd.read_html(url)[17]
    df4 = df4.drop([0,2,4,6]).drop(df4.columns[[0,1,5,6]], axis=1)
    #トラッキングデータ
    df5 = pd.read_html(url)[20]
    df5 = df5.drop([0,2,4,6]).drop(df5.columns[[0,1,5,6]], axis=1)

    #他のスタッツ
    df6 = pd.read_html(url)[35]
    df6 = df6.drop([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34]).drop(df6.columns[[0,1,5,6]], axis=1)

    df = pd.concat([df,df3,df4,df5,df6])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'試合平均': 'home','Unnamed: 3': 'Data','試合平均.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)
    #チーム名と試合日
    homeTeam = get_keys_from_value(TeamDic,homeTeamID)
    awayTeam = get_keys_from_value(TeamDic,awayTeamID)
    df.insert(0, 'チーム名',[homeTeam,awayTeam])
    df.insert(0, '試合日', matchDate)
    data = pd.concat([data, df]) 
    data['総移動距離'] = data['総移動距離'].str.replace('km', '').astype(np.float64)
    data['相手との移動距離差'] = data['相手との移動距離差'].str.replace('km', '').astype(np.float64)
    data['チャンス構築率'] = data['チャンス構築率'].str.replace('%', '').astype(np.float64)
    data['成功率'] = data['成功率'].str.replace('%', '').astype(np.float64)
    data['被成功率'] = data['被成功率'].str.replace('%', '').astype(np.float64)
    data['被チャンス構築率'] = data['被チャンス構築率'].str.replace('%', '').astype(np.float64)

    return data

In [ ]:
pred_data = scrapePredData(pred_team['Ｇ大阪'],pred_team['FC東京'],'2021','2021-8-21-19:00')
pred_data

Data,試合日,チーム名,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート,チャンス構築率,ゴール,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール,被ゴール期待値,被ゴール,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール,シュート,枠内シュート,パス,クロス,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
home,2021-8-21-19:00,[Ｇ大阪],14.48,11.95,1.35,1.19,5.27,1.68,87.60,14.24,0.68,114.2,10.5,9.2,0.7,7.1,114.4,14.8,13.0,1.0,6.7,1.118,0.7,1.532,1.0,115.112,-0.513,160.2,-8.8,0.7,10.5,3.2,502.8,13.0,10.2,2.0,4.2,20.3,9.6,17.1,21.8,1.8,1.6,0.9,0.0,37.5,9.8
away,2021-8-21-19:00,[FC東京],13.58,11.04,1.44,1.1,6.11,3.8,90.11,14.05,0.34,119.3,11.0,9.3,1.3,12.2,118.6,13.3,11.2,1.3,10.2,1.145,1.3,1.34,1.3,111.544,-2.040,189.2,11.7,1.3,11,3.4,409.9,11.7,12.3,1.2,4.9,22.3,10.3,18.4,19.7,1.2,1.3,1,0,33.5,11


# SetUP

In [ ]:
from pycaret.regression import setup
data = data.drop('チーム名', axis=1)
data = data.drop('Unnamed: 0',axis = 1)
setup(data, target="スコア",
       numeric_features=["退場"],
        session_id = 123)

,Description,Value
0,session_id,123
1,Target,スコア
2,Original Data,"(454, 47)"
3,Missing Values,False
4,Numeric Features,45
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(317, 64)"


({'cooks': 'Cooks Distance',
  'error': 'Prediction Error',
  'feature': 'Feature Importance',
  'feature_all': 'Feature Importance (All)',
  'learning': 'Learning Curve',
  'manifold': 'Manifold Learning',
  'parameter': 'Hyperparameters',
  'residuals': 'Residuals',
  'residuals_interactive': 'Interactive Residuals',
  'rfe': 'Feature Selection',
  'tree': 'Decision Tree',
  'vc': 'Validation Curve'},
 {'ada': <pycaret.containers.models.regression.AdaBoostRegressorContainer at 0x7f76e25f6190>,
  'ard': <pycaret.containers.models.regression.AutomaticRelevanceDeterminationContainer at 0x7f76e25ac410>,
  'br': <pycaret.containers.models.regression.BayesianRidgeContainer at 0x7f76e25acd50>,
  'dt': <pycaret.containers.models.regression.DecisionTreeRegressorContainer at 0x7f76e25f6110>,
  'en': <pycaret.containers.models.regression.ElasticNetContainer at 0x7f76e25ac7d0>,
  'et': <pycaret.containers.models.regression.ExtraTreesRegressorContainer at 0x7f76e25f8390>,
  'gbr': <pycaret.contai

In [ ]:
data.dtypes

試合日             object
スコア              int64
攻撃             float64
パス             float64
クロス            float64
ドリブル           float64
シュート           float64
ゴール            float64
奪取             float64
守備             float64
セーブ            float64
攻撃回数           float64
シュート.1         float64
チャンス構築率        float64
ゴール.1          float64
成功率            float64
被攻撃回数          float64
被シュート          float64
被チャンス構築率       float64
被ゴール           float64
被成功率           float64
ゴール期待値         float64
ゴール.2          float64
被ゴール期待値        float64
被ゴール.1         float64
総移動距離          float64
相手との移動距離差      float64
スプリント数         float64
相手とのスプリント数差    float64
ゴール.3          float64
シュート.2         float64
枠内シュート         float64
パス.1           float64
クロス.1          float64
直接ＦＫ           float64
間接ＦＫ           float64
ＣＫ             float64
スローイン          float64
ドリブル.1         float64
タックル           float64
クリア            float64
インターセプト        float64
オフサイド          float64
警告         

In [ ]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
br,Bayesian Ridge,7.985000e-01,1.072300e+00,1.028500e+00,6.770000e-02,0.0,3.405000e-01,0.017
en,Elastic Net,8.007000e-01,1.086100e+00,1.034600e+00,5.770000e-02,0.0,3.358000e-01,0.017
lasso,Lasso Regression,8.058000e-01,1.094700e+00,1.037700e+00,5.580000e-02,0.0,3.317000e-01,0.015
omp,Orthogonal Matching Pursuit,8.241000e-01,1.075600e+00,1.033100e+00,5.130000e-02,0.0,4.065000e-01,0.016
lightgbm,Light Gradient Boosting Machine,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.014
ada,AdaBoost Regressor,8.561000e-01,1.146100e+00,1.067500e+00,-2.520000e-02,0.0,3.789000e-01,0.101
llar,Lasso Least Angle Regression,8.564000e-01,1.191600e+00,1.083900e+00,-2.740000e-02,0.0,3.372000e-01,0.015
ridge,Ridge Regression,8.651000e-01,1.190100e+00,1.086300e+00,-5.510000e-02,0.0,4.410000e-01,0.014
huber,Huber Regressor,8.540000e-01,1.189600e+00,1.085300e+00,-5.760000e-02,0.0,4.159000e-01,0.047
lr,Linear Regression,8.731000e-01,1.207200e+00,1.094000e+00,-7.370000e-02,0.0,4.542000e-01,0.292


In [ ]:
print(best_model)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)


# Create Model

In [ ]:
from pycaret.regression import models

models()  # 機械学習モデルの一覧

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [ ]:
from pycaret.regression import create_model

br = create_model("br")  # Bayesian Ridgeのモデルを作成

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.7908,1.0609,1.0300,0.1037,-0.0,0.3117
1,0.6481,0.8525,0.9233,0.2468,-0.0,0.2534
2,0.9484,1.5734,1.2544,0.1004,-0.0,0.3851
3,0.7722,0.9278,0.9632,0.0722,-0.0,0.3301
4,0.8891,1.1666,1.0801,-0.1587,-0.0,0.4962
5,0.9591,1.2707,1.1273,-0.1074,-0.0,0.3934
6,0.8084,1.2348,1.1112,0.0916,-0.0,0.3364
7,0.6325,0.7678,0.8762,0.1928,-0.0,0.2337
8,0.6680,0.7121,0.8439,-0.0093,-0.0,0.3110
9,0.8681,1.1562,1.0753,0.1453,-0.0,0.3538


In [ ]:
print(br)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)


# Tune Model

In [ ]:
from pycaret.regression import tune_model

tuned_br = tune_model(br)  # ハイパーパラメータの調整

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.8056,1.0072,1.0036,0.1490,-0.0,0.3409
1,0.6912,0.9073,0.9525,0.1984,-0.0,0.3000
2,0.9439,1.5310,1.2374,0.1246,-0.0,0.4030
3,0.8076,0.9803,0.9901,0.0197,-0.0,0.3694
4,0.8597,1.0657,1.0323,-0.0585,-0.0,0.4873
5,0.9650,1.2277,1.1080,-0.0699,-0.0,0.4209
6,0.7837,1.1768,1.0848,0.1343,-0.0,0.3414
7,0.7005,0.8400,0.9165,0.1168,-0.0,0.3048
8,0.6885,0.7445,0.8629,-0.0553,-0.0,0.3259
9,0.8665,1.1317,1.0638,0.1634,-0.0,0.3534


In [ ]:
print(tuned_br)

BayesianRidge(alpha_1=0.1, alpha_2=0.1, alpha_init=None, compute_score=True,
              copy_X=True, fit_intercept=True, lambda_1=0.01, lambda_2=1e-06,
              lambda_init=None, n_iter=300, normalize=True, tol=0.001,
              verbose=False)


# evalation

In [ ]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_br)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# deploy

In [ ]:
from pycaret.regression import finalize_model

final_br = finalize_model(tuned_br)
print(final_br)

BayesianRidge(alpha_1=0.1, alpha_2=0.1, alpha_init=None, compute_score=True,
              copy_X=True, fit_intercept=True, lambda_1=0.01, lambda_2=1e-06,
              lambda_init=None, n_iter=300, normalize=True, tol=0.001,
              verbose=False)


In [ ]:
from pycaret.regression import predict_model

predict_model(final_br);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Bayesian Ridge,0.9005,1.2813,1.132,0.127,0.5354,0.3628


In [ ]:
from pycaret.regression import save_model

save_model(final_br, "br_model")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['退場'], target='スコア',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  BayesianRidge(alpha_1=0.1, alpha_2=0.1, alpha_init=None,
 

In [ ]:
from pycaret.regression import load_model

loaded_br = load_model("br_model")

Transformation Pipeline and Model Successfully Loaded


# prediction

In [ ]:
pred_data

Data,試合日,チーム名,攻撃,パス,クロス,ドリブル,シュート,ゴール,奪取,守備,セーブ,攻撃回数,シュート,チャンス構築率,ゴール,成功率,被攻撃回数,被シュート,被チャンス構築率,被ゴール,被成功率,ゴール期待値,ゴール,被ゴール期待値,被ゴール,総移動距離,相手との移動距離差,スプリント数,相手とのスプリント数差,ゴール,シュート,枠内シュート,パス,クロス,直接ＦＫ,間接ＦＫ,ＣＫ,スローイン,ドリブル,タックル,クリア,インターセプト,オフサイド,警告,退場,３０ｍライン進入,ペナルティエリア進入
home,2021-8-21-19:00,[Ｇ大阪],14.48,11.95,1.35,1.19,5.27,1.68,87.60,14.24,0.68,114.2,10.5,9.2,0.7,7.1,114.4,14.8,13.0,1.0,6.7,1.118,0.7,1.532,1.0,115.112,-0.513,160.2,-8.8,0.7,10.5,3.2,502.8,13.0,10.2,2.0,4.2,20.3,9.6,17.1,21.8,1.8,1.6,0.9,0.0,37.5,9.8
away,2021-8-21-19:00,[FC東京],13.58,11.04,1.44,1.1,6.11,3.8,90.11,14.05,0.34,119.3,11.0,9.3,1.3,12.2,118.6,13.3,11.2,1.3,10.2,1.145,1.3,1.34,1.3,111.544,-2.040,189.2,11.7,1.3,11,3.4,409.9,11.7,12.3,1.2,4.9,22.3,10.3,18.4,19.7,1.2,1.3,1,0,33.5,11


In [ ]:
#pred_data = pred_data.drop('チーム名', axis=1)
#pred_data = pred_data.drop('Data', axis=1)
prediction = predict_model(loaded_br, data=pred_data)
prediction

KeyError: ignored

In [ ]:
data_unseen